In [1]:
# token by transformer multi-lingual
import torch
#from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer, BertConfig, BertForTokenClassification
from transformers import AutoModelWithLMHead, AutoTokenizer
#from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification
#from transformers import pipeline, AdamW
from transformers import BertTokenizer, BertModel, BertConfig

In [2]:
MAX_LEN = 75
BATCH_SIZE = 32

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
config = BertConfig.from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)

In [79]:
text = "오에 겐자부로는 일본 현대문학의 초석을 놓은 것으로 평가받는 작가 나쓰메 소세키(1867~1916)의 대표작 ‘마음’에 담긴 군국주의적 요소, 야스쿠니 신사 참배 행위까지 소설의 삽화로 동원하며 일본 사회의 ‘비정상성’을 문제 삼는다."

In [9]:
# tokenize
# return tokens
print(tokenizer.tokenize(text))
# token vectorize
# ret = {'input_ids': **, 'token_type_ids': **, 'attention_mask': **}
print(tokenizer.encode_plus(text)) # args: pad_to_max_length 

['오', '##에', '겐', '##자', '##부', '##로는', '일본', '현대', '##문', '##학', '##의', '초', '##석', '##을', '놓', '##은', '것으로', '평', '##가', '##받', '##는', '작', '##가', '나', '##쓰', '##메', '소', '##세', '##키', '(', '1867', '~', '1916', ')', '의', '대', '##표', '##작', '[UNK]', '마', '##음', '[UNK]', '에', '담', '##긴', '군', '##국', '##주의', '##적', '요', '##소', ',', '야', '##스', '##쿠', '##니', '신', '##사', '참', '##배', '행', '##위', '##까지', '소', '##설', '##의', '삽', '##화', '##로', '동', '##원', '##하며', '일본', '사', '##회의', '[UNK]', '비', '##정', '##상', '##성', '[UNK]', '을', '문', '##제', '삼', '##는다', '.']
{'input_ids': [101, 9580, 10530, 8873, 13764, 14646, 70186, 23130, 104518, 25934, 23321, 10459, 9757, 40958, 10622, 9029, 10892, 23925, 9926, 11287, 118965, 11018, 9652, 11287, 8982, 119103, 118927, 9448, 24982, 21039, 113, 13821, 198, 11785, 114, 9637, 9069, 37824, 38709, 100, 9246, 32158, 100, 9559, 9064, 70221, 8910, 20479, 37224, 14801, 9599, 22333, 117, 9538, 12605, 61156, 25503, 9487, 12945, 9735, 76036, 9966, 19855, 18382, 9448, 3

In [13]:
# padding (separate case with transformers)
# use pad_sequences
# get input of list
from keras.preprocessing.sequence import pad_sequences
tag_temp1 = tokenizer.encode_plus(text)['input_ids'][:5]
tag_temp2 = tokenizer.encode_plus(text)['input_ids'][5:10]
tag_temp = [tag_temp1, tag_temp2]
tags_test = pad_sequences(tag_temp, maxlen=10, value=-1, \
                     padding="post", dtype="long", truncating="post")
print(tags_test)

[[   101   9580  10530   8873  13764     -1     -1     -1     -1     -1]
 [ 14646  70186  23130 104518  25934     -1     -1     -1     -1     -1]]


In [15]:
# SKT KoBERT
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import BertTokenizer, BertModel, BertConfig

In [16]:
# get_pytorch_kobert_model gives model and vocab
kobert, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [48]:
# KoBERT tokenizer
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)

using cached model


In [67]:
text

'오에 겐자부로는 일본 현대문학의 초석을 놓은 것으로 평가받는 작가 나쓰메 소세키(1867~1916)의 대표작 ‘마음’에 담긴 군국주의적 요소, 야스쿠니 신사 참배 행위까지 소설의 삽화로 동원하며 일본 사회의 ‘비정상성’을 문제 삼는다.'

In [66]:
tokens = tokenizer(text)
print(tokens)

['▁오', '에', '▁', '겐', '자', '부', '로', '는', '▁일본', '▁현대', '문학', '의', '▁초', '석', '을', '▁놓', '은', '▁것으로', '▁평가', '받는', '▁작가', '▁나', '쓰', '메', '▁소', '세', '키', '▁(', '▁18', '67', '▁', '~', '▁19', '16', '▁', ')', '▁', '의', '▁대표', '작', '▁‘', '▁마음', '▁', '’', '▁', '에', '▁담긴', '▁군', '국', '주의', '적', '▁요소', '▁', ',', '▁야', '스', '쿠', '니', '▁신', '사', '▁참', '배', '▁행위', '까지', '▁소설', '의', '▁', '삽', '화', '로', '▁동원', '하며', '▁일본', '▁사회', '의', '▁‘', '▁비', '정', '상', '성', '▁', '’', '▁', '을', '▁문제', '▁삼', '는', '다', '▁', '.']


In [70]:
texts = text.split()
ret = []
for text1 in texts:
    ret.extend(tokenizer(text1))
print(ret)
ret == tokens

['▁오', '에', '▁', '겐', '자', '부', '로', '는', '▁일본', '▁현대', '문학', '의', '▁초', '석', '을', '▁놓', '은', '▁것으로', '▁평가', '받는', '▁작가', '▁나', '쓰', '메', '▁소', '세', '키', '▁(', '▁18', '67', '▁', '~', '▁19', '16', '▁', ')', '▁', '의', '▁대표', '작', '▁‘', '▁마음', '▁', '’', '▁', '에', '▁담긴', '▁군', '국', '주의', '적', '▁요소', '▁', ',', '▁야', '스', '쿠', '니', '▁신', '사', '▁참', '배', '▁행위', '까지', '▁소설', '의', '▁', '삽', '화', '로', '▁동원', '하며', '▁일본', '▁사회', '의', '▁‘', '▁비', '정', '상', '성', '▁', '’', '▁', '을', '▁문제', '▁삼', '는', '다', '▁', '.']


True

In [72]:
# KoBERT vectorizer
# :return:
# ids, length, token_type_id
tok_transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=30, pair=False)
# KoBERT defines mask separately 

In [80]:
import numpy as np
def get_attention_mask(len_sent, valid_length):
        attention_mask = np.zeros(len_sent)
        attention_mask[:valid_length] = 1
        return attention_mask
input_ids, length, type_id = tok_transform((text,))
attention_mask = get_attention_mask(len(input_ids), length)
print(input_ids)
print(length)
print(type_id)
print(attention_mask)

[   2 3417 6896  517 5402 7147 6398 6079 5760 3809 5051 6237 7095 4501
 6557 7088 1522 7086  909 4842 6290 3931 1370 6779 6190 2822 6579 7573
  522    3]
30
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


In [62]:
## test
#from gluonnlp.data import SentencepieceTokenizer, SentencepieceDetokenizer
#tok2 = SentencepieceTokenizer(get_tokenizer())

using cached model


In [65]:
tok2('신지애-최경주 부진한 출발')

['▁신', '지', '애', '-', '최', '경', '주', '▁부진', '한', '▁출발']

In [74]:
text

'오에 겐자부로는 일본 현대문학의 초석을 놓은 것으로 평가받는 작가 나쓰메 소세키(1867~1916)의 대표작 ‘마음’에 담긴 군국주의적 요소, 야스쿠니 신사 참배 행위까지 소설의 삽화로 동원하며 일본 사회의 ‘비정상성’을 문제 삼는다.'

In [77]:
text = "첫 회를 시작으로 13일까지 4일간 총 4회에 걸쳐 매 회 2편씩 총 "

In [ ]:
class KoBERT():
    ## wrap